In this notebook, I use dash and other component to create the Covid_19 dashboard. The data is from [Our World in Data](https://github.com/owid/covid-19-data/tree/master/public/data). The dashboard replicates some of the features of the COVID-19 dashboard by the [WHO](https://covid19.who.int/).

It is recommended to create an virtual environment using Anaconda prompt to avoid the conflict when loading new components. 

In [ ]:
# loading packages. These packages should be installed in advance when creating new virtual environment. 
# Using either conda install or pip install 
import dash
from dash import dcc, html
from jupyter_dash import JupyterDash
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc

from dash_bootstrap_templates import load_figure_template

import plotly.express as px
import pandas as pd
from datetime import datetime, timedelta

In [ ]:
# set URL to external CSS file 
dbc_css = 'https://cdn.jsdelivr.net/gh/AnnMarieW/dash-bootstrap-templates@V1.0.2/dbc.min.css'

## 1. Extract and wrangle data

In [ ]:
# load data, the data is updated from 24 Feb 2020 to now.
url = 'https://covid.ourworldindata.org/data/owid-covid-data.csv'

# create a list with name of some varaibles from the dataset
cols_lst = ['iso_code', 
            'location', 
            'continent',
            'date', 
            'new_cases', 
            'total_cases',
            'total_cases_per_million',
            'new_deaths', 
            'total_deaths',
            'total_deaths_per_million',
            'people_fully_vaccinated',
            'total_vaccinations_per_hundred',
            'people_vaccinated_per_hundred',
            'new_vaccinations']

# create the dataframe using pandas read_csv, columns' names are as cols_lst
df = pd.read_csv(url, usecols = cols_lst) # 205692 obs × 14 variables

# Convert date to datetime
df['date'] = pd.to_datetime(df['date'], format = '%Y-%m-%d')

df.head()

In [ ]:
# first count the total NaN observations.
df.isna().sum()

In [ ]:
# there are too many NaN obs, I assume that the three columns new_deaths, new_cases, and new_vaccinations 
# could be 0. 
for col in ['new_vaccinations', 'new_deaths', 'new_cases']:
    df[col].fillna(0, inplace=True)

In [ ]:
# for continent, it also contains aggregates, aka continents. 
# df[df['continent'].isna()] # 11893 obs
# I filter the observations for the world and store it in a variable called df_world
# extract wolrd
df_world = df[df['location'] == 'World'].copy() # 923 obs
df_world.tail()

First feature, using df_world to extract the global totals. 

In [ ]:
df_world[df_world['people_fully_vaccinated'].isna()] # 326 obs

In [ ]:
df_world_tot = df_world.groupby('location').last().copy() # group the world input, using groupby().last() function. 
# only data from last row displays.

# extract total cases
tot_deaths = int(df_world_tot['total_deaths'])
tot_cases = int(df_world_tot['total_cases'])
tot_vac = int(df_world_tot['people_fully_vaccinated'])

print('Total deaths: {:,}'.format(tot_deaths))
print('Total cases: {:,}'.format(tot_cases))
print('People fully vaccinated: {:,}'.format(tot_vac))

Feature 2, using df to extract the most recent data for each location in the world

In [ ]:
# extract the most recent obs, not NaN, for each location.
df_last = df.groupby('location').last().reset_index() # 244 obs
df_last.head()

In [ ]:
# drop aggregates (missing continent info)
df_last = df_last[~df_last['continent'].isna()].copy()
print('Number of unique countries: ', df_last['location'].nunique())

In [ ]:
df_last.isna().sum()
# there are still some contries that missings info 
df_last[df_last['total_deaths'].isna()]
# but they seem like small islands/ countiries with smalls number of covid19 cases. so we can ignore these missing data

Feature 3, using df_world but now for calcualting the weekly sums of new deaths , cases and vaccinations.

In [ ]:
# set index to date 
df_world.set_index('date', inplace = True)

# use resample method to sum each colum by week
df_world_week = df_world.resample('W').sum().reset_index() # 133 obs


In [ ]:
df_world_week.head()

Creating a function call plot_bar that returns a bar plot of a given column in df_world_week

In [ ]:
def plot_bar(col_name, data = df_world_week):
    ''' define a function to return a figure. it has two params
    col_name: the name of column we want to plot
    data = df_world_week: set the df_world_week as the default
    return: the bar chart that illustrates the col_name data'''
    
    # create the bar chart with x-axis as date, y-axis as col_name values.
    fig = px.bar(
        data, 
        x = 'date',
        y = col_name,
        labels = {'date': 'Date', col_name: 'Week sum'})
    
    # set the layout for the bar chart 
    fig.update_layout(
        xaxis_title = None, 
        yaxis_title = None, 
        margin = {'t': 0, 'b': 0, 'r': 0, 'l': 0},
        height = 300
    )
    return fig

plot_bar('new_vaccinations')

Creating a function call plot_map using df_last that returns a world map

In [ ]:
def plot_map(col_name, data= df_last):
    '''define a function that return a world map, 
    params col_name: name of the column from df_last,
    param df_last: set the df_last as the default dataset
    return: a world map with col_name data display'''
    
    # set a world map
    fig = px.choropleth(
        data, 
        locations = 'iso_code', 
        color = col_name, 
        scope = 'world', 
        hover_name = 'location', 
        hover_data = {'iso_code': False}, 
        labels = {col_name: 'value'}
    )
    
    # update world map layout that all right, left, top, bottom = 0
    fig.update_layout(margin = {'r': 0, 'l': 0, 't': 0, 'b': 0})
    
    fig.update_layout(
        coloraxis_colorbar_title_text = None, 
        coloraxis_showscale = True, 
        geo = {'showframe': False})
    
    return fig 

plot_map('new_vaccinations')

## 2. Create the dashboard

In [ ]:
# set the total deaths card
deaths_card = dbc.Card(
    children = [ 
        html.H4('Total deaths'),
        html.H2('{:,}'.format(tot_deaths))
    ],
    body = True,)

# set the total cases card
cases_card = dbc.Card(
    children = [ 
        html.H4('Total cases'),
        html.H2('{:,}'.format(tot_cases))
    ],
    body = True,)

# set the total vaccinantion card
vac_card = dbc.Card(
    children = [ 
        html.H4('People vaccinated'),
        html.H2('{:,}'.format(tot_vac))
    ],
    body = True,)

Feature 2 & 3, creating two selectors, a `Dropdown` component for selecting the variable and a `Dropdown` component for selecting the metric in feature 2. The two selectors are placed inside a `Card` component.

In [ ]:
# select variables 
var_selector = dcc.Dropdown(
    id = 'variable',
    options = [{'label': 'Deaths', 'value': 'deaths'},
               {'label': 'Cases', 'value': 'cases'},
               {'label': 'Vaccinations', 'value': 'vaccinations'}],
    value = 'deaths',
    clearable = False,
)

# metric selector 
metric_selector = dcc.Dropdown(
    id = 'metric', 
    options = [], 
    value = None, 
    clearable = False, 
    optionHeight = 80
    )

In [ ]:
selector_card = dbc.Card([
    html.Label('Select variable: '),
    var_selector,
    html.Br(),
    html.Label('Select metric: '), 
    metric_selector
],
    body = True, color = 'primary')

In [ ]:
# place feature 2 & 3 inside Card components 
map_card = dbc.Card([
    dbc.Row([
        dbc.Col(id = 'current_week', width = 3),
        dbc.Col(dcc.Graph(id = 'bar_plot', config = {'displayModeBar': False}), width = 9)
    ])
])

In [ ]:
bar_card = dbc.Card([
    dbc.Row([
        dbc.Col(selector_card, width = 3),
        dbc.Col(dcc.Graph(id = 'map', config = {'displayModeBar': False}), width = 9)
    ])
], 
    body =True
)

In [ ]:
template = 'lux'
load_figure_template(template)
app = JupyterDash(external_stylesheets = [dbc.themes.LUX, dbc_css])

description = """
A Dash application that tracks the development of COVID-19 and vaccination around the world. 

Data is collected from [Our World in Data](https://ourworldindata.org/coronavirus).
"""

app.layout = dbc.Container([
    
    html.H1('COVID-19 tracker'),
    html.P(dcc.Markdown(description)),
    
    # Row wth cards with totals
    dbc.Row(
        children = [
            dbc.Col(deaths_card, width = 4),
            dbc.Col(cases_card, width = 4),
            dbc.Col(vac_card, width = 4)
        ]
    ),
    html.Br(),
    
    # Card with selectors and map
    map_card,
    html.Br(),
    
    # Card with bar plot
    bar_card,
    html.Br()
    
],   
    className = 'dbc'
)

@app.callback(
    Output('metric', 'options'),
    Output('metric', 'value'),
    Input('variable', 'value')
)
def set_metric_options(var):
    
    if var == 'vaccinations':
        options = [
            {'label' : 'Total doses administered per 100 population', 'value' : 'total_vaccinations_per_hundred'},
            {'label' : 'Persons vaccinated with at least one dose per 100 population', 'value' : 'people_vaccinated_per_hundred'},
            {'label' : 'Persons fully vaccinated with last dose of primary series', 'value' : 'people_fully_vaccinated'}
        ]
        value = 'total_vaccinations_per_hundred'
        
    else:
        options = [
            {'label' : 'Total', 'value' : 'total'},
            {'label' : 'Total per 1 million population', 'value' : 'total_per_million'},
            {'label' : 'Newely reported in last 24 hours', 'value' : 'last_24h'},
        ]
        value = 'total'

    return options, value

@app.callback(
    Output('map', 'figure'),
    Input('variable', 'value'),
    Input('metric', 'value')
)
def update_map(var, metric):
    
    if metric == 'last_24h':
        col_name = 'new_' + var
    elif metric == 'total':
        col_name = 'total_' + var
    elif metric == 'total_per_million':
        col_name = 'total_' + var + '_per_million'
    else:
        col_name = metric
        
    fig = plot_map(col_name)
    
    return fig


@app.callback(
    Output('bar_plot', 'figure'),
    Output('current_week', 'children'),
    Input('variable', 'value')
)
def update_bar(var, df = df_world_week):
    
    col_name = 'new_' + var
    
    # Create figure
    fig = plot_bar(col_name)
    fig.update_layout(
        xaxis_title = None, 
        yaxis_title = None, 
        margin =  {'t' : 0, 'b' : 0, 'r' : 0, 'l' : 0},
        height = 250
    ) 
    
    # Create text
    current_number = int(df.iloc[-2][col_name])
    text = dbc.Container([
        html.H2('{:,}'.format(current_number)),
        html.H4(var),
        html.H4('last week')
    ], style = {'marginTop' : 50})
    
    return fig, text
    

app.run_server()